In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase

tavily_client = TavilyClient()

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")


@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [3]:
from dataclasses import dataclass

@dataclass
class UserRole:
    user_role: str = "external"

In [5]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Dynamically call tools based on the runtime context"""

    user_role = request.runtime.context.user_role
    
    if user_role == "internal":
        pass # internal users get access to all tools
    else:
        tools = [web_search] # external users only get access to web search
        request = request.override(tools=tools) 

    return handler(request)

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search, sql_query],
    middleware=[dynamic_tool_call],
    context_schema=UserRole
)

In [7]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "external"}
)

print(response["messages"][-1].content)

I don’t have access to your database, so I can’t see the exact count. If you tell me the database type and schema, I can give you the exact query. In the meantime, here are common ways to count artists:

- SQL (assuming a table named artists):
  - Total artists (one row per artist): SELECT COUNT(*) FROM artists;
  - Artists who have at least one work (assuming a works table with artist_id): SELECT COUNT(DISTINCT artist_id) FROM works;

- SQL (counting unique artist names in the artists table):
  - SELECT COUNT(DISTINCT name) FROM artists;

- MongoDB (collection named artists):
  - db.artists.countDocuments({});
  - Alternatively: db.artists.distinct("_id").length

- Python with pandas (if you have a dataframe of artists):
  - len(artists_df['artist_id'].dropna().unique())

If you share your schema (table/collection names, relevant fields) and the database type (MySQL, PostgreSQL, MongoDB, etc.), I’ll give you the exact query. If you’d like, you can paste a snippet of your schema or sam

In [8]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "internal"}
)

print(response["messages"][-1].content)

There are 275 artists in the database (in the Artist table).

If you’d like, I can show the first few artists or export the list.
